In this section, I will present you the final view factor calculator which is optimiezed from Nusselt Analogue method and Monte Carlo method.

In [51]:
import matplotlib.pyplot as plt
from t4gpd.io.CirReader import CirReader
from t4gpd.pyvista.ToUnstructuredGrid import ToUnstructuredGrid
from shapely.geometry import Polygon
from geopandas import GeoDataFrame
import numpy as np
import pandas as pd
import geopandas
from math import *
from numpy.random import randint
from shapely.geometry import LineString,Point
from t4gpd.commons.GeomLib import GeomLib
from t4gpd.demos.GeoDataFrameDemos import GeoDataFrameDemos
from t4gpd.morph.geoProcesses.FootprintExtruder import FootprintExtruder
from t4gpd.morph.geoProcesses.STGeoProcess import STGeoProcess
from t4gpd.morph.STPointsDensifier2 import STPointsDensifier2
from t4gpd.pyvista.ToUnstructuredGrid import ToUnstructuredGrid
import random
from t4gpd.commons.RayCasting3Lib import RayCasting3Lib
import timeit
from matplotlib.gridspec import GridSpec
from pandas import concat,merge
from t4gpd.pyvista.commons.RayCasting3DLib import RayCasting3DLib
from t4gpd.pyvista.geoProcesses.RayCasting3D import RayCasting3D
from t4gpd.pyvista.STRaysToViewFactors import STRaysToViewFactors
from t4gpd.pyvista.geoProcesses.MoveSensorsAwayFromSurface import MoveSensorsAwayFromSurface
from t4gpd.morph.STGrid import STGrid

# General Function

In [2]:
def SurfaceNormal(self): #input is the buildingsIn3d.geometry[i],than choose the polygon of one facet
    coord = self.exterior.coords
    facet_p0=np.array(coord[0])
    facet_p1=np.array(coord[1])
    facet_p2=np.array(coord[2])
    vec1 = facet_p1 - facet_p0
    vec2 = facet_p2 - facet_p1
    a = vec1[1]*vec2[2]-vec1[2]*vec2[1]
    b = vec1[2]*vec2[0]-vec1[0]*vec2[2]
    c = vec1[0]*vec2[1]-vec1[1]*vec2[0]
    normal = np.array([a,b,c])
    mod = module(normal)
    n  = normal/mod
    return n

In [3]:
def module(vector):
    return np.sqrt(np.dot(np.array(vector),np.array(vector)))

In [4]:
def VectorAngleCos(vector1,vector2):#calculate the angle between two vectors
    unit_vector1 = vector1/np.linalg.norm(vector1)
    unit_vector2 = vector2/np.linalg.norm(vector2)
    dot_vector = np.dot(unit_vector1,unit_vector2)
    return dot_vector

In [5]:
def STEvenPoints(face,n,m): #row=n, column=m
    p = face.exterior.coords
    vec1 = np.array(p[2])-np.array(p[1])
    vec2 = np.array(p[1])-np.array(p[0])
    L1 = vec1/n
    L2 = vec2/m
    s = []
    normal = []
    for i in range(1,n+1):
        for j in range(1,m+1):
            point_1 = np.array(p[0])+L2*(j-1)+L1*(i-1)
            point_2 = np.array(p[0])+L2*j+L1*i
            point_c = Point((point_1+point_2)/2)
            s.append(point_c)
            normal.append(SurfaceNormal(face))
    p_geodata = GeoDataFrame({'geometry':s},crs='epsg:2154')
    p_geodata['NormalVector'] = normal
    return p_geodata

In [7]:
def centroid(self): #rectangle centroid
    coord = self.exterior.coords
    face_p0=np.array(coord[0])
    face_p2=np.array(coord[2])
    face_ce=(face_p0+face_p2)/2
    return Point(face_ce)

In [8]:
def grid3d(self,n,m):#n rows, m columns
    coord = self.exterior.coords
    p0=np.array(coord[0])
    p1=np.array(coord[1])
    p2=np.array(coord[2])
    p3=np.array(coord[3])
    vec1 = (p1-p0)/m
    vec2 = (p2-p1)/n
    poly=[]
    cen_p=[]
    sur_area=[]
    normal=[]
    sur_d=[]
    for i in range(n):
        for j in range(m):
            p0_t=Point(p0+j*vec1+i*vec2)
            p1_t=Point(p0+(j+1)*vec1+i*vec2)
            p2_t=Point(p0+(j+1)*vec1+(i+1)*vec2)
            p3_t=Point(p0+j*vec1+(i+1)*vec2)
            face_t = Polygon([p0_t,p1_t,p2_t,p3_t,p0_t])
            poly.append(face_t)
            cen_p.append(centroid(face_t))
            sur_area.append(area3d(face_t))
            normal.append(SurfaceNormal(face_t))
            sur_d.append(Surface_diameter(face_t))
    self_grid=GeoDataFrame({'geometry': poly},crs='epsg:2154')
    self_grid['CentroidPoint']=cen_p
    self_grid['area']=sur_area
    self_grid['NormalVector']=normal
    self_grid['diameter']=sur_d
    return self_grid

In [9]:
def Surface_diameter(surface):
    coord = surface.exterior.coords
    d = distance3d(Point(coord[0]),Point(coord[2]))
    return d

In [11]:
def distance3d(point1,point2):
    d = sqrt((point1.x-point2.x)**2 + (point1.y-point2.y)**2 + (point1.z-point2.z)**2)
    return d

In [13]:
#get area from 3d polygon
def area3d(self):
    coord = self.exterior.coords
    d1 = distance3d(Point(coord[0]),Point(coord[1]))
    d2 = distance3d(Point(coord[1]),Point(coord[2]))
    return d1*d2

In [22]:
def OnSurface(self,point):
    ang = 0
    p = np.array(point)
    coord = self.exterior.coords
    for i in range(len(coord)-1):
        vec1 = np.array(coord[i]) - p
        vec2 = np.array(coord[i+1])-p
        m1 = module(vec1)
        m2 = module(vec2)
        if cmp(m1*m2,0):
            return True
        else:
            return False

In [19]:
def cmp(a,b,c=1e-3):
    return abs(a-b)<c

In [26]:
def IFshareedge(f1,f2):
    coord1 = f1.geometry.exterior.coords
    coord2 = f2.geometry.exterior.coords
    for i in coord1:
        if OnSurface(f2.geometry,i):
            return True
    for j in coord2:
        if OnSurface(f1.geometry,j):
            return True
    return False

# Nusselt Analogue Sphere

In [16]:
def isFiveTimeRuleOk(f1,f2):
    return distance3d(f1.CentroidPoint,f2.CentroidPoint) >= 5*f1.diameter

In [15]:
#Subdivide function 2
def Subdivide2(f1,f2): #f1 is the emitting surface, f2 is the target surface
    if isFiveTimeRuleOk(f1,f2):
        return [f1]
    else:
        d = distance3d(f1.CentroidPoint,f2.CentroidPoint)/5
        coord1 = f1.geometry.exterior.coords
        w1 = distance3d(Point(coord1[0]),Point(coord1[1]))
        h1 = distance3d(Point(coord1[1]),Point(coord1[2]))
        
        n1 = int(h1/d)+1
        m1 = int(w1/d)+1
        
        x = grid3d(f1.geometry,n1,m1)
        return x

In [445]:
def vf_nusselt(face1,face2): #emitting from face1 accept by face2
    pt1 = face1.CentroidPoint
    pt2 = face2.CentroidPoint
    n1 = face1.NormalVector
    n2 = face2.NormalVector
    vec = np.array(pt2)-np.array(pt1)
    a2 = face2.area
    r = distance3d(pt1,pt2)
    vf = -a2*VectorAngleCos(n1,vec)*VectorAngleCos(n2,vec)/(pi*r**2)
    if vf>0:
        return vf
    else:
        return 0

# Monte Carlo Ray Tracing

In [6]:
def SelectRay(vec0,vec1,ray,method):#transform into 2d
    if method=='horizontal':
        v0 = np.array([vec0[0],vec0[1]])#here v0 is the left point
        v1 = np.array([vec1[0],vec1[1]])
        r = np.array([ray[0],ray[1]])
    if method=='vertical':
        v0 = np.array([vec0[1],vec0[2]])#here v0 is the taller point
        v1 = np.array([vec1[1],vec1[2]])
        r = np.array([ray[1],ray[2]])

    x = v0[0]*r[1]-v0[1]*r[0]
    y = v1[0]*r[1]-v1[1]*r[0]
    if x<=0 and y>=0:
        return True
    else:
        return False

In [12]:
def RandomDir(n):#number of samples
    vector = []
    for i in range(n):
        polar = 0.5*np.arccos(1-2*random.random())
        azimuth = random.random()*pi*2
        x = cos(polar)*sin(azimuth)
        y = sin(polar)
        z = cos(polar)*cos(azimuth)
        vector.append(np.array([x,y,z]))
    return vector

In [10]:
def RayPoint2(point,n,norm,length):
    ray_line = []
    vec = RandomDir(n)
    for i in range(len(vec)):
        vec1 = vec[i]
        x2 = norm[0]*vec1[1]+norm[1]*vec1[0]
        y2 = norm[1]*vec1[1]-norm[0]*vec1[0]
        z2 = vec1[2]
        p2 = Point(point.x+x2*length,point.y+y2*length,point.z+z2*length)
        ray_vec = np.array([p2.x,p2.y,p2.z]) - np.array([point.x,point.y,point.z])
        ray_line.append(ray_vec)
    return ray_line

In [76]:
def newMonteCarlo(f1,f2,nb_ray,n,m,times):
    g1 = grid3d(f1.geometry,n,m)
    g2 = grid3d(f2.geometry,n,m)
    masks = concat([g1,g2])
    masks.reset_index(inplace=True, drop=True)
    masks['pk'] = masks.index
    
    sensors = GeoDataFrame({'geometry':list(masks.CentroidPoint)},crs='epsg:2154')
    sensors['NormalVector'] = list(masks.NormalVector)
    sensors['pk'] = list(masks.pk)
    op1 = MoveSensorsAwayFromSurface(sensors,normalFieldname='NormalVector',dist=1e-10)
    sensors = STGeoProcess(op1,sensors).execute()
    
    viewfactor = []
    
    coords = f2.geometry.exterior.coords#selectray
    
    for w in range(times):
        vf = []
        #shootingDirs = RayCasting3DLib.preparePanopticRays(nb_ray)
        ray = RayPoint2(sensors.geometry.iloc[0],nb_ray,sensors.NormalVector.iloc[0],1)
        for i in range(int(len(sensors)/2)):
            sensor = sensors.loc[ sensors[sensors.pk == i].index ]
            
            #select ray
            point = np.array(sensor.geometry.squeeze())
            vec0 = np.array(coords[0])-point
            vec1 = np.array(coords[1])-point
            vec2 = np.array(coords[2])-point
            ray_vec = []
            for j in range(len(ray)):
                if SelectRay(vec2,vec0,ray[j],'vertical')==True:
                    ray_vec.append(ray[j])
            shootingDirs = np.asarray(ray_vec)
            
            #calculate viewfactor
            op2 = RayCasting3D([masks], shootingDirs, viewpoints=sensor,
                      normalFieldname='NormalVector', pkFieldname='pk')
            rays = STGeoProcess(op2, sensor).execute()
            vfSparseMatrix = STRaysToViewFactors(rays,'pk','hitGids').run()
            vf.append(sum(vfSparseMatrix.viewfactor)*len(ray_vec)/len(ray))
        viewfactor.append(sum(vf)/(len(sensors)/2))
    return np.mean(viewfactor)

# View Factor Calculator

In [426]:
def ViewFactor(f1,f2):
    if isFiveTimeRuleOk(f1,f2):
        vf_nu = vf_nusselt(f1,f2)
        return vf_nu
    elif IFshareedge(f1,f2)==False:
        d = GeoDataFrame(Subdivide2(f1,f2))
        c = GeoDataFrame(Subdivide2(f2,f1))
        vf2_nu = 0
        for i in range(len(d)):
            face_0 = d.iloc[i]
            vf2 = [vf_nusselt(face_0,face_1) for face_1 in c.iloc()]
            vf2_nu = vf2_nu + sum(vf2)/len(d)
        return vf2_nu
    else:
        vf_mc = newMonteCarlo(f1,f2,10000,3,3,1)
        return vf_mc

# Create Surfaces

In [20]:
def CreateFaces_share(theta): #x=w/d, y=h/d
    polygon1 = []
    polygon2 = []
    coords_3d_1 = np.array([[10,0,0], [0,0,0], [0,0,10], [10,0,10], [10,0,0]])
    coords_3d_2 = np.array([[0,0,0], [10,0,0], [10,10*sin(theta),10*cos(theta)], [0,10*sin(theta),10*cos(theta)], [0,0,0]])

    polygon1.append(Polygon(coords_3d_1))
    polygon2.append(Polygon(coords_3d_2))
    f1 = GeoDataFrame({'geometry':polygon1},crs='epsg:2154')
    f2 = GeoDataFrame({'geometry':polygon2},crs='epsg:2154')
    
    f1['area'] = area3d(f1.geometry.iloc[0])
    f2['area'] = area3d(f2.geometry.iloc[0])
    
    f1['diameter'] = Surface_diameter(f1.geometry.iloc[0])
    f2['diameter'] = Surface_diameter(f2.geometry.iloc[0])
    
    f1['CentroidPoint'] = centroid(f1.geometry.iloc[0])
    f2['CentroidPoint'] = centroid(f2.geometry.iloc[0])
    return f1,f2

In [28]:
def CreateFaces(x,y,method): #x=w/d, y=h/d
    k = y/x
    polygon1 = []
    polygon2 = []
    coords_3d_1 = np.array([[10,0,0], [0,0,0], [0,0,10*k], [10,0,10*k], [10,0,0]])
    if method == 'parallel':
        coords_3d_2 = np.array([[0,10/x,0], [10,10/x,0], [10,10/x,10*k], [0,10/x,10*k], [0,10/x,0]])
    elif method == 'vertical':
        coords_3d_2 = np.array([[0,0,0], [10,0,0], [10,10/x,0], [0,10/x,0], [0,0,0]])
    else:
        return 'Please select your method "parallel" or "vertical"?'
    polygon1.append(Polygon(coords_3d_1))
    polygon2.append(Polygon(coords_3d_2))
    f1 = GeoDataFrame({'geometry':polygon1},crs='epsg:2154')
    f2 = GeoDataFrame({'geometry':polygon2},crs='epsg:2154')
    
    f1['area'] = area3d(f1.geometry.iloc[0])
    f2['area'] = area3d(f2.geometry.iloc[0])
    
    f1['diameter'] = Surface_diameter(f1.geometry.iloc[0])
    f2['diameter'] = Surface_diameter(f2.geometry.iloc[0])
    
    f1['CentroidPoint'] = centroid(f1.geometry.iloc[0])
    f2['CentroidPoint'] = centroid(f2.geometry.iloc[0])
    
    n1=[]
    n2=[]
    n1.append(SurfaceNormal(f1.geometry.iloc[0]))
    n2.append(SurfaceNormal(f2.geometry.iloc[0]))
    f1['NormalVector'] = n1
    f2['NormalVector'] = n2
    return f1,f2

# 3D mockup

In [211]:
from t4gpd.demos.GeoDataFrameDemos5 import GeoDataFrameDemos5

In [212]:
gdf = GeoDataFrameDemos5.cirSceneMasque1Corr() #different mockups with cirSceneMasque1Corr and cirSceneMasque2Corr

In [213]:
#create lables
lab=[]
roof_x=[]
roof_y=[]
roof_z=[]
for i in gdf.geometry:
    fig = Polygon(i)
    x, y, z=zip(*fig.exterior.coords)
    if np.std(z)==0 and np.mean(z)==0:
        lab.append('ground')
    elif np.std(z)==0 and np.mean(z)!=0:
        lab.append('roof')
        roof_x.append(x)
        roof_y.append(y)
        roof_z.append(z)
    else:
        lab.append('floor')
gdf['lable'] = lab
gdf['gid'] = gdf.index

In [214]:
#find normal vector
normal_vec = []
for i in range(len(gdf)):
    if gdf.lable[i]=='roof' or gdf.lable[i]=='ground':
        normal_vec.append(np.array([0,0,1]))
    else:
        normal_vec.append(SurfaceNormal(gdf.geometry[i]))
normal_vec
gdf['NormalVector'] = normal_vec

In [215]:
#create 2d buildings geodataframe
geo2d=[]
height=[]
for i in range(len(roof_x)):
    p = Polygon(list(zip(roof_x[i],roof_y[i])))
    geo2d.append(p)
    height.append(np.mean(roof_z[i]))
buildings = GeoDataFrame({'geometry':geo2d},crs='epsg:2154')
buildings['HAUTEUR'] = height

In [216]:
#create ground with holes
_ground = gdf.loc[gdf.lable=='ground'].geometry
_footprint = gdf.loc[gdf.lable=='roof'].geometry
for i in _footprint:
    _ground = _ground.difference(i)
gdf.loc[gdf.lable=='ground','geometry']=_ground.squeeze()

In [221]:
#create ground grid
dx = 5
ggeo=[]
ggeo.append(gdf.loc[gdf.lable=='ground'].geometry.iloc[0])
ground = GeoDataFrame({'geometry':ggeo},crs='epsg:2154')
grid = STGrid(ground, dx, dy=None, indoor=True, intoPoint=False, encode=False).run()
grid = grid.drop(columns=['neighbors4','neighbors8','indoor','gid'])
grid['lable'] = ['ground' for item in grid.index]
grid['NormalVector'] = [np.array([0,0,1]) for item in grid.index]
#grid.gid = grid.index
for i in range(len(grid)):
    grid.geometry[i] = GeomLib.forceZCoordinateToZ0(grid.geometry[i],z0=0)
grid['diameter'] = [Surface_diameter(face_g) for face_g in grid.geometry]
grid['CentroidPoint'] = [centroid(face_g) for face_g in grid.geometry]
grid['area'] = [area3d(face_g) for face_g in grid.geometry]

In [408]:
def Addproperty(self):
    self['NormalVector'] = [SurfaceNormal(item) for item in self.geometry]
    self['CentroidPoint'] = [centroid(item) for item in self.geometry]
    self['diameter'] = [Surface_diameter(item) for item in self.geometry]
    self['area'] = [area3d(item) for item in self.geometry]
    la = []
    for i in self.CentroidPoint:
        if i.z==0:
            la.append('ground')
        else:
            la.append('floor')
    self['lable'] = la
    return self

In [218]:
#create floor grid
floor_grid = GeoDataFrame(columns=['geometry','CentroidPoint','area','NormalVector','diameter'])
for i in range(len(gdf)):
    if gdf.lable[i]=='floor':
        coord = gdf.geometry.iloc[i].exterior.coords
        w = distance3d(Point(coord[0]),Point(coord[1]))
        h = distance3d(Point(coord[1]),Point(coord[2]))
        if w<h:
            m = int(h/3)
            g = grid3d(gdf.geometry.iloc[i],m,1)
        else:
            m = int(w/3)#height of one floor
            g = grid3d(gdf.geometry.iloc[i],1,m)
        floor_grid = floor_grid.append(g)
floor_grid = floor_grid.reset_index(drop=True)
floor_grid['lable'] = ['floor' for item in floor_grid.index]

In [402]:
#concat both floor grids and ground grids
allfaces = pd.concat([floor_grid,grid])
allfaces = allfaces.reset_index(drop=True)

In [403]:
#Covisible facing surface
def FaceSurface(s_surface,t_surface):
    min_d = 1e-8
    n1 = s_surface.NormalVector
    n2 = t_surface.NormalVector
    coords = t_surface.geometry.exterior.coords
    point = np.array(s_surface.CentroidPoint)
    p_s = s_surface.CentroidPoint
    p_t = t_surface.CentroidPoint
    p_t = Point(p_t.x+n2[0]*min_d,p_t.y+n2[1]*min_d,p_t.z+n2[2]*min_d)
    
    covisible, ray = RayCasting3Lib.areCovisibleIn3D(p_s, p_t, buildings, 'HAUTEUR')
    if covisible:
        for j in range(1,len(coords)):
            vec = np.array(coords[j])-point
            con1 = vec.dot(n1)
            con2 = vec.dot(n2)
            if con1>0 and con2<0:
                return True
    else:
        return False

In [464]:
%%time
#find facing and covisible surfaces from test grid
index_test = 477
testgrid = GeoDataFrame.copy(allfaces.loc[(allfaces.index==index_test)])
ff=[]
for i in range(len(allfaces)):
    if FaceSurface(testgrid.iloc[0],allfaces.iloc[i]):
        ff.append(allfaces.iloc[i])
covisible_grid = GeoDataFrame(ff)
covisible_grid = Addproperty(covisible_grid)
covisible_grid = covisible_grid.reset_index(drop=True)

#calculate view factor
vf = [ViewFactor(testgrid.iloc[0],co_grid) for co_grid in covisible_grid.iloc]
covisible_grid['ViewFactor'] = vf

CPU times: total: 5.67 s
Wall time: 5.67 s


In [469]:
scene1 = ToUnstructuredGrid([testgrid,covisible_grid],'ViewFactor').run()
scene1.plot(scalars='ViewFactor',cmap='gist_earth',show_edges=True,
	show_scalar_bar=True, point_size=5.0, render_points_as_spheres=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)